# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import torch
import torch.nn as nn
from numpy.random.mtrand import permutation

In [2]:
print(torch.backends.mps.is_available())
device = torch.device("cpu") # Defaults to CPU

True


In [3]:

# Example: Move a tensor or model to the MPS device
class TwoBranch(nn.Module):
    def __init__(self):
        super().__init__()
        model = nn.LSTM(input_size=3, hidden_size=16, batch_first=False, dropout=0.2, num_layers=1).to(device)
        self.left = model
        self.right = model
        self.combine = nn.Linear(32, 32).to(device)
        nn.init.xavier_uniform_(model.weight_ih_l0)
        nn.init.xavier_uniform_(model.weight_hh_l0)
    def forward(self, x_left, x_right):
        _, (l_h, _) = self.left(x_left)
        _, (r_h, _) = self.right(x_right)

        l = l_h[-1]  # last layer, shape: (batch, hidden_size)
        r = r_h[-1]

        cat = torch.cat([l, r], dim=-1)  # shape: (batch, 10)
        return self.combine(cat)          # shape: (batch, 10) -> Linear(10,1) -> (batch,1)
class BigModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.branch = TwoBranch()
        self.rest = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        ).to(device)

    def forward(self, x_left, x_right):
        x = self.branch(x_left, x_right)
        return self.rest(x)
model2=BigModel().to(device)



/Users/everettwilber/PyCharmMiscProject/.venv/lib/python3.13/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


In [4]:
"""
batch = 32
seq_len = 64

x_left  = torch.randn(seq_len, batch, 3, device=device)
x_right = torch.randn(seq_len, batch, 3, device=device)

# labels depend on problem:
y = torch.randn(batch, 1, device=device)  # regression
"""

'\nbatch = 32\nseq_len = 64\n\nx_left  = torch.randn(seq_len, batch, 3, device=device)\nx_right = torch.randn(seq_len, batch, 3, device=device)\n\n# labels depend on problem:\ny = torch.randn(batch, 1, device=device)  # regression\n'

In [5]:
import pandas as pd
import glob
files = glob.glob("data/*.json")
people = []
for file in files:
    df = pd.read_json(file)
    drawings = []
    for drawing in df.values:
        drawing = drawing[0]
        allTriples = []
        for stroke in drawing:
            points = stroke["points"]
            triples = [(point["x"], point["y"], point["time"]) for point in points]
            allTriples.extend(triples)
        drawings.append(allTriples)
    people.append(drawings)


data = people


In [6]:
import json
from pathlib import Path

out_path = Path("output/data.json")

out_path.write_text(json.dumps(data, indent=2))

983106

In [7]:
out_path = Path("output/data.json")
loaded = json.load(out_path.open())

In [8]:
import numpy as np

training: list[list[list[tuple[int, int, int]]]] = loaded
pairedDeep = [[torch.tensor(drawing, device=device) for drawing in drawings] for drawings in training]
paired: list[torch.Tensor] = []
personIDs: list[int] = []
for personID, person in enumerate(pairedDeep):
    person = [drawing[:-1]-drawing[1:] for drawing in person]
    a = [personID] * len(person)
    personIDs.extend(a)
    paired.extend([((i[:, :] - i.min(dim=0, keepdim=True).values) / (
    (i.max(dim=0, keepdim=True).values - i.min(dim=0, keepdim=True).values)))*2-0.5 for i in person])
personIDs = np.array(personIDs)
lengths = torch.tensor([len(s) for s in paired])


def packPad(pairs: list[torch.Tensor], lengths: torch.Tensor, indeces):
    padded = nn.utils.rnn.pad_sequence([pairs[i].to(device) for i in indeces], batch_first=True).to(device)
    lengths = lengths[indeces]
    packed = nn.utils.rnn.pack_padded_sequence(
        padded,
        lengths,
        batch_first=True,
        enforce_sorted=False
    ).float().to(device)
    return packed



# Split at PERSON level to prevent any leakage
n_people = len(pairedDeep)
split_person_idx = int(n_people * 0.8)

# Get all indices for train people and test people
train_person_indices = []
test_person_indices = []

for person_id in range(n_people):
    person_mask = personIDs == person_id
    person_samples = np.where(person_mask)[0]

    if person_id < split_person_idx:
        train_person_indices.extend(person_samples)
    else:
        test_person_indices.extend(person_samples)

train_person_indices = np.array(train_person_indices)
test_person_indices = np.array(test_person_indices)

print(f"Training people: 0-{split_person_idx-1}")
print(f"Test people: {split_person_idx}-{n_people-1}")
print(f"Train samples: {len(train_person_indices)}")
print(f"Test samples: {len(test_person_indices)}")


def generate_balanced_pairs(indices, n_pairs, personIDs):
    """Generate 50/50 same-person vs different-person pairs"""
    left_indices = []
    right_indices = []

    n_same = n_pairs // 2
    n_diff = n_pairs - n_same

    # Same-person pairs
    for _ in range(n_same):
        idx = np.random.choice(indices)
        person_id = personIDs[idx]
        same_person_mask = personIDs[indices] == person_id
        same_person_indices = indices[same_person_mask]

        if len(same_person_indices) > 1:
            pair = np.random.choice(same_person_indices, size=2, replace=False)
            left_indices.append(pair[0])
            right_indices.append(pair[1])
        else:
            left_indices.append(idx)
            right_indices.append(idx)

    # Different-person pairs
    for _ in range(n_diff):
        idx1 = np.random.choice(indices)
        person1 = personIDs[idx1]
        diff_person_mask = personIDs[indices] != person1
        diff_person_indices = indices[diff_person_mask]

        if len(diff_person_indices) > 0:
            idx2 = np.random.choice(diff_person_indices)
            left_indices.append(idx1)
            right_indices.append(idx2)
        else:
            left_indices.append(idx1)
            right_indices.append(np.random.choice(indices))

    return np.array(left_indices), np.array(right_indices)


# Generate balanced pairs
n_train_pairs = len(train_person_indices)  # Reduce multiplier for stability
n_test_pairs = len(test_person_indices)

pLT, pRT = generate_balanced_pairs(train_person_indices, n_train_pairs, personIDs)
pLTest, pRTest = generate_balanced_pairs(test_person_indices, n_test_pairs, personIDs)

pairedNP = paired
lengthsNP = lengths
leftTrain = packPad(pairedNP, lengthsNP, pLT).to(device)
rightTrain = packPad(pairedNP, lengthsNP, pRT).to(device)
yTrain = (torch.tensor(personIDs[pLT] == personIDs[pRT])
          .reshape((-1, 1))
          .to(device, dtype=torch.float32))

leftTest = packPad(pairedNP, lengthsNP, pLTest).to(device)
rightTest = packPad(pairedNP, lengthsNP, pRTest).to(device)
yTest = (torch.tensor(personIDs[pLTest] == personIDs[pRTest])
         .reshape((-1, 1))
         .to(device, dtype=torch.float32))

print(f"\nTrain: {(yTrain == 1).sum().item() / len(yTrain) * 100:.1f}% same person")
print(f"Test:  {(yTest == 1).sum().item() / len(yTest) * 100:.1f}% same person")


Training people: 0-15
Test people: 16-20
Train samples: 214
Test samples: 63

Train: 50.0% same person
Test:  49.2% same person


In [29]:
len(np.unique(personIDs[train_person_indices])), len(np.unique(personIDs[test_person_indices]))


(16, 5)

In [9]:
str(((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean().item())

'0.4920634925365448'

In [10]:
model2.branch.left.all_weights

[[Parameter containing:
  tensor([[-0.0638, -0.2254,  0.0629],
          [ 0.2376,  0.0336,  0.0846],
          [ 0.0508, -0.0449, -0.1388],
          [ 0.1512,  0.0095,  0.2152],
          [ 0.2858,  0.0678,  0.0285],
          [-0.1612,  0.0790, -0.2730],
          [ 0.0517, -0.2180,  0.1074],
          [ 0.1704, -0.1956,  0.1989],
          [-0.0285, -0.2683,  0.2919],
          [ 0.1163,  0.1361,  0.0430],
          [-0.1374, -0.1551,  0.0709],
          [-0.0355, -0.0658, -0.2429],
          [-0.2852,  0.0573,  0.1404],
          [-0.2975, -0.0958,  0.1661],
          [-0.0181,  0.0750, -0.0700],
          [ 0.2024, -0.1251, -0.0781],
          [-0.2292,  0.2341, -0.1329],
          [-0.0795,  0.1985, -0.0054],
          [-0.2805, -0.1088,  0.1123],
          [-0.2612,  0.1200,  0.0562],
          [-0.1628,  0.1457,  0.0859],
          [-0.0462,  0.2351, -0.0880],
          [ 0.0075,  0.1968, -0.0495],
          [ 0.0902, -0.1011,  0.1015],
          [-0.2144, -0.2722, -0.0633],
 

In [11]:
epochs = 400
# BCELogits with L2 reg
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.AdamW(model2.parameters(), lr=0.001, weight_decay=1e-5)


In [12]:
annealer = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
for epoch in range(epochs):
    pLTBatch, pRTBatch = generate_balanced_pairs(train_person_indices, n_train_pairs, personIDs)
    leftTrainBatch = packPad(pairedNP, lengthsNP, pLTBatch).to(device)
    rightTrainBatch = packPad(pairedNP, lengthsNP, pRTBatch).to(device)
    yTrainBatch = (torch.tensor(personIDs[pLTBatch] == personIDs[pRTBatch])
          .reshape((-1, 1))
          .to(device, dtype=torch.float32))

    epoch_loss = 0.0
    for i in range(4):
        optimizer.zero_grad()
        outputBatch = model2(leftTrainBatch, rightTrainBatch)
        lossBatch = criterion(outputBatch, yTrainBatch)

        lossBatch.backward()
        optimizer.step()
        optimizer.zero_grad()
        epoch_loss += lossBatch.item()
    annealer.step()
    epoch_loss /= 4

    # average loss for the epoch
    if epoch % 25 != 0 and epoch % 5 == 0:
        print(f"Epoch {epoch:3d} – loss: {epoch_loss:.4f} - fPC {((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean():.4f}, {((model2(leftTrain, rightTrain)>0)==(yTrain>0.5)).float().mean():.4f}")
    if epoch % 25 == 0:
        output = model2(leftTrain, rightTrain)
        loss = criterion(output, yTrain)
        print(f"Epoch {epoch:3d} – loss: {loss:.4f} - fPC {((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean():.4f}, {((model2(leftTrain, rightTrain)>0)==(yTrain>0.5)).float().mean():.4f}")
        if loss.item() < 0.59:
            break
output = model2(leftTrain, rightTrain)
loss = criterion(output, yTrain)
assert loss.item() <= 0.6 # if this assertion fails that means we got stuck and the model was unable to find the right strategy.
model2

Epoch   0 – loss: 0.6932 - fPC 0.5079, 0.5000
Epoch   5 – loss: 0.6925 - fPC 0.4444, 0.5234
Epoch  10 – loss: 0.6926 - fPC 0.4603, 0.5093
Epoch  15 – loss: 0.6897 - fPC 0.4921, 0.5187
Epoch  20 – loss: 0.6842 - fPC 0.7143, 0.5514
Epoch  25 – loss: 0.6745 - fPC 0.7619, 0.5794
Epoch  30 – loss: 0.6452 - fPC 0.8254, 0.6449
Epoch  35 – loss: 0.6183 - fPC 0.6667, 0.6168
Epoch  40 – loss: 0.5916 - fPC 0.6508, 0.6822
Epoch  45 – loss: 0.4863 - fPC 0.7778, 0.7664
Epoch  50 – loss: 0.5064 - fPC 0.8730, 0.7243


BigModel(
  (branch): TwoBranch(
    (left): LSTM(3, 16, dropout=0.2)
    (right): LSTM(3, 16, dropout=0.2)
    (combine): Linear(in_features=32, out_features=32, bias=True)
  )
  (rest): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [13]:
output = model2(leftTrain, rightTrain)
loss = criterion(output, yTrain)
print(f"Epoch {epoch:3d} – loss: {loss:.4f} - fPC {((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean():.4f}, {((model2(leftTrain, rightTrain)>0)==(yTrain>0.5)).float().mean():.4f}")

Epoch  50 – loss: 0.5064 - fPC 0.8730, 0.7243


In [14]:
# if "pretrainedModel" in globals().keys():
#     model2 = pretrainedModel
# else:
#     pretrainedModel = model2


In [15]:
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.SGD(model2.parameters(), lr=0.0001, weight_decay=1e-4, momentum=0.9)

In [16]:
annealer = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=500)

In [20]:
for epoch in range(500-170):
    permute = np.random.permutation(train_person_indices.shape[0])[:8]
    pLTBatch, pRTBatch = generate_balanced_pairs(train_person_indices[permute], n_train_pairs*8, personIDs)
    leftTrainBatch = packPad(pairedNP, lengthsNP, pLTBatch).to(device)
    rightTrainBatch = packPad(pairedNP, lengthsNP, pRTBatch).to(device)
    yTrainBatch = (torch.tensor(personIDs[pLTBatch] == personIDs[pRTBatch])
          .reshape((-1, 1))
          .to(device, dtype=torch.float32))

    optimizer.zero_grad()
    epoch_loss = 0.0
    outputBatch = model2(leftTrainBatch, rightTrainBatch)
    lossBatch = criterion(outputBatch, yTrainBatch)

    lossBatch.backward()
    optimizer.step()
    annealer.step()
    optimizer.zero_grad()

    epoch_loss += lossBatch.item()

    # average loss for the epoch
    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d} – loss: {epoch_loss:.4f} - fPC {((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean():.4f}, {((model2(leftTrain, rightTrain)>0)==(yTrain>0.5)).float().mean():.4f}")



print(f"Epoch {epoch:3d} – loss: {epoch_loss:.4f} - fPC {((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean():.4f}, {((model2(leftTrain, rightTrain)>0)==(yTrain>0.5)).float().mean():.4f}")

Epoch   0 – loss: 0.5225 - fPC 0.8730, 0.7430
Epoch  10 – loss: 0.3658 - fPC 0.8730, 0.7430
Epoch  20 – loss: 0.3665 - fPC 0.8730, 0.7430
Epoch  30 – loss: 0.4738 - fPC 0.8730, 0.7477
Epoch  40 – loss: 0.4321 - fPC 0.8730, 0.7523
Epoch  50 – loss: 0.3906 - fPC 0.8730, 0.7523
Epoch  60 – loss: 0.3546 - fPC 0.8730, 0.7570
Epoch  70 – loss: 0.3546 - fPC 0.8730, 0.7664
Epoch  80 – loss: 0.3582 - fPC 0.8571, 0.7710
Epoch  90 – loss: 0.3613 - fPC 0.8571, 0.7710
Epoch 100 – loss: 0.6325 - fPC 0.8571, 0.7710
Epoch 110 – loss: 0.3346 - fPC 0.8571, 0.7710
Epoch 120 – loss: 0.3881 - fPC 0.8571, 0.7710
Epoch 130 – loss: 0.4077 - fPC 0.8571, 0.7710
Epoch 140 – loss: 0.3540 - fPC 0.8571, 0.7710
Epoch 150 – loss: 0.2632 - fPC 0.8571, 0.7757
Epoch 160 – loss: 0.2901 - fPC 0.8571, 0.7757
Epoch 170 – loss: 0.3505 - fPC 0.8571, 0.7757
Epoch 180 – loss: 0.4576 - fPC 0.8571, 0.7757
Epoch 190 – loss: 0.4915 - fPC 0.8571, 0.7757
Epoch 200 – loss: 0.5994 - fPC 0.8571, 0.7757
Epoch 210 – loss: 0.2842 - fPC 0.8

In [21]:
model = model2

In [ ]:
model2 = model

In [22]:
ckpt = {
    'epoch'         : epoch,                     # current epoch number
    'model_state'   : model.state_dict(),        # model parameters
    'optimizer_state': optimizer.state_dict(),   # optimizer internals
    'scheduler_state': annealer.state_dict() if annealer else None,
}
torch.save(ckpt, 'bad.pt')
torch.save(model.state_dict(), 'bad.pt')


In [23]:
import torch

def confusion_rates(yhat: torch.Tensor, y: torch.Tensor):
    tp = (yhat == 1) & (y == 1)
    tn = (yhat == 0) & (y == 0)
    fp = (yhat == 1) & (y == 0)
    fn = (yhat == 0) & (y == 1)

    TP = int(tp.sum().item())
    TN = int(tn.sum().item())
    FP = int(fp.sum().item())
    FN = int(fn.sum().item())

    eps = 1e-12
    TPR = TP / (TP + FN + eps)   # recall / sensitivity
    TNR = TN / (TN + FP + eps)   # specificity
    FPR = FP / (FP + TN + eps)   # 1 - specificity
    FNR = FN / (FN + TP + eps)   # 1 - recall

    return {"TP": TP, "TN": TN, "FP": FP, "FN": FN,
            "TPR": TPR, "TNR": TNR, "FPR": FPR, "FNR": FNR}

def metrics(model, leftTest, rightTest, yTest):
    yhat = model(leftTest, rightTest) > 0
    y = yTest
    return confusion_rates(yhat, y)

res = metrics(model2, leftTest, rightTest, yTest)

print(f"True Positive Rate (Recall) : {res['TPR']:.4f}; TP : {res['TP']}")
print(f"True Negative Rate (Spec.)  : {res['TNR']:.4f}; TN : {res['TN']}")
print(f"False Positive Rate         : {res['FPR']:.4f}; FP : {res['FP']}")
print(f"False Negative Rate         : {res['FNR']:.4f}; FN : {res['FN']}")


True Positive Rate (Recall) : 0.9032; TP : 28
True Negative Rate (Spec.)  : 0.8437; TN : 27
False Positive Rate         : 0.1562; FP : 5
False Negative Rate         : 0.0968; FN : 3


In [26]:
device = torch.device("cpu") # Defaults to CPU

# Example: Move a tensor or model to the MPS device
class TwoBranchFinal(nn.Module):
    def __init__(self):
        super().__init__()
        model = nn.LSTM(input_size=3, hidden_size=16, batch_first=False, num_layers=1).to(device)
        self.left = model
        self.right = model
        self.combine = nn.Linear(32, 32).to(device)
        nn.init.xavier_uniform_(model.weight_ih_l0)
        nn.init.xavier_uniform_(model.weight_hh_l0)
    def forward(self, x_left, x_right):
        _, (l_h, _) = self.left(x_left)
        _, (r_h, _) = self.right(x_right)

        l = l_h[-1]  # last layer, shape: (batch, hidden_size)
        r = r_h[-1]

        cat = torch.cat([l, r], dim=-1)  # shape: (batch, 10)
        return self.combine(cat)          # shape: (batch, 10) -> Linear(10,1) -> (batch,1)
class BigModelProd(nn.Module):
    def __init__(self):
        super().__init__()
        self.branch = TwoBranchFinal()
        self.rest = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        ).to(device)

    def forward(self, x_left, x_right):
        x = self.branch(x_left, x_right)
        return self.rest(x)

weights = torch.load('model_weights_other.pt', map_location=device)
model_new = BigModelProd()          # the same class you used during training
model_new.load_state_dict(weights)
model_new.to(device)

res = metrics(model_new, leftTest, rightTest, yTest)
print("Testing:")
print(f"True Positive Rate (Recall) : {res['TPR']:.4f}; TP : {res['TP']}")
print(f"True Negative Rate (Spec.)  : {res['TNR']:.4f}; TN : {res['TN']}")
print(f"False Positive Rate         : {res['FPR']:.4f}; FP : {res['FP']}")
print(f"False Negative Rate         : {res['FNR']:.4f}; FN : {res['FN']}")
print()
res = metrics(model_new, leftTrain, rightTrain, yTrain)
print("Training:")
print(f"True Positive Rate (Recall) : {res['TPR']:.4f}; TP : {res['TP']}")
print(f"True Negative Rate (Spec.)  : {res['TNR']:.4f}; TN : {res['TN']}")
print(f"False Positive Rate         : {res['FPR']:.4f}; FP : {res['FP']}")
print(f"False Negative Rate         : {res['FNR']:.4f}; FN : {res['FN']}")

positive: np.ndarray = (nn.Sigmoid()(model_new(leftTest, rightTest) - yTest)).detach().numpy()[yTest == 1]
negative: np.ndarray = (nn.Sigmoid()(model_new(leftTest, rightTest) - yTest)).detach().numpy()[yTest == 0]
positive.resize(max(negative.size,positive.size))
negative.resize(max(positive.size,negative.size))
np.vstack((positive,negative))
positive,negative

Testing:
True Positive Rate (Recall) : 0.9032; TP : 28
True Negative Rate (Spec.)  : 0.8437; TN : 27
False Positive Rate         : 0.1562; FP : 5
False Negative Rate         : 0.0968; FN : 3

Training:
True Positive Rate (Recall) : 0.8411; TP : 90
True Negative Rate (Spec.)  : 0.6916; TN : 74
False Positive Rate         : 0.3084; FP : 33
False Negative Rate         : 0.1589; FN : 17


(array([0.48004085, 0.5337593 , 0.14064959, 0.5476826 , 0.36310938,
        0.58609   , 0.51609075, 0.49388075, 0.49752975, 0.14486243,
        0.4368893 , 0.5609226 , 0.21539004, 0.5913554 , 0.5947775 ,
        0.5718898 , 0.4359781 , 0.3527774 , 0.3677233 , 0.4473823 ,
        0.5610101 , 0.5550844 , 0.49440593, 0.49419335, 0.49625412,
        0.5657231 , 0.57466877, 0.43665236, 0.60352755, 0.5613196 ,
        0.52599514, 0.        ], dtype=float32),
 array([0.1445959 , 0.3156685 , 0.04014008, 0.10604444, 0.1441207 ,
        0.0355303 , 0.02916823, 0.09374581, 0.3203415 , 0.02792595,
        0.3885638 , 0.05924111, 0.05008363, 0.01962832, 0.7284317 ,
        0.18959895, 0.0685721 , 0.0461051 , 0.7114167 , 0.63278234,
        0.12396618, 0.46992308, 0.20911328, 0.12874149, 0.03917939,
        0.18748778, 0.40572485, 0.04698633, 0.6819311 , 0.36078992,
        0.58779305, 0.18767975], dtype=float32))